# COCHES DE SEGUNDA MANO



### ¿Qué se quiere predecir?

El precio de venta del coche de segunda mano
### ¿Qué se puede explicar?

Factores que afectan el precio: ¿Cuáles son las características que más influyen en el precio? <br>
Tendencias del mercado: ¿Qué marcas/modelos se devalúan más rápido? <br>
Patrones de demanda: ¿Qué tipo de coches se venden más rápido? <br>

In [ ]:

import pandas as pd
from tabulate import tabulate

df = pd.read_csv("datasets/used_car_dataset.csv")

if 'Unnamed: 0' in df.columns:
    df = df.set_index('Unnamed: 0')

# Selecciona y ordena columnas (ajusta a lo que tengas)
cols = [c for c in [
    'brand', 'model_date', 'price', 'currency',
    'fuel_type', 'vehicle_engine', 'vehicle_transmission',
    'mileage_from_odometer', 'item_condition', 'manufacturer', 'description'
] if c in df.columns]
df = df[cols]

print(tabulate(df.head(10), headers='keys', tablefmt='fancy_grid', showindex=False))


╒═════════╤══════════════╤═══════════╤════════════╤═════════════╤══════════════════╤════════════════════════╤═════════════════════════╤══════════════════╤════════════════╤════════════════════════════════════════════╕
│ brand   │   model_date │     price │ currency   │ fuel_type   │ vehicle_engine   │ vehicle_transmission   │ mileage_from_odometer   │ item_condition   │ manufacturer   │ description                                │
╞═════════╪══════════════╪═══════════╪════════════╪═════════════╪══════════════════╪════════════════════════╪═════════════════════════╪══════════════════╪════════════════╪════════════════════════════════════════════╡
│ MG      │         2021 │ 6.525e+06 │ PKR        │ Petrol      │ 1490cc           │ Automatic              │ 6,000 km                │ used             │ MG             │ MG HS 2021 for sale in Lahore              │
├─────────┼──────────────┼───────────┼────────────┼─────────────┼──────────────────┼────────────────────────┼───────────────────────

## ¿Como diferenciar si el problema es de regresión, clasificación o clustering?

### Pregunta clave inicial

| Pregunta de negocio                                   | Tipo de problema                               | Ejemplo de target / salida                       |
|-------------------------------------------------------|------------------------------------------------|--------------------------------------------------|
| ¿Cuál será el precio estimado de este coche?          | Regresión                                      | `price` (numérico continuo)                     |
| ¿En qué categoría de precio cae (barato / medio / alto)? | Clasificación multiclase (derivada)          | `price_segment` (feature creado)                |
| ¿Se venderá rápido (sí / no)?                         | Clasificación binaria                          | `fast_sale` (derivado de días de publicación)   |
| ¿Qué grupos naturales de coches existen?              | Clustering (no supervisado)                    | (Sin target)                                    |
| ¿Qué coches tienen precios anómalos?                  | Detección de anomalías (relacionado a clustering) | (Sin target)                                   |

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
import numpy as np

df = pd.read_csv("datasets/used_car_dataset.csv")
df = df[df['price'].notna()]

if 'model_date' in df.columns:
    df['model_date'] = pd.to_numeric(df['model_date'], errors='coerce')
    df['vehicle_age'] = datetime.now().year - df['model_date']

features = ['brand','mileage_from_odometer','vehicle_age','fuel_type',
            'vehicle_transmission','item_condition']
features = [f for f in features if f in df.columns]

X = df[features]
y = pd.to_numeric(df['price'], errors='coerce')
mask = y.notna()
X, y = X[mask], y[mask]

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

pre = ColumnTransformer([
    ('num','passthrough', num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

model = Pipeline([
    ('prep', pre),
    ('reg', RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42))
])

X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_tr, y_tr)
pred = model.predict(X_val)
mae = mean_absolute_error(y_val, pred)
print(f"MAE: {mae:,.2f}")

MAE: 1,161,902.35
